In [1]:
%load_ext autoreload
%autoreload 2
import warnings
from os import path

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, patches
import matplotlib.gridspec as gridspec
import seaborn as sns
from tqdm.auto import tqdm
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tqdm.pandas()
from sklearn.decomposition import PCA

from tbd_eeg.data_analysis.eegutils import *
from tbd_eeg.data_analysis.Utilities import utilities as utils
from tbd_eeg.data_analysis.Utilities import filters

import differentiation

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False
%matplotlib widget

In [2]:
# accessing the Google sheet with experiment metadata in python
# setting up the permissions:
# 1. install gspread (pip install gspread / conda install gspread)
# 2. copy the service_account.json file to '~/.config/gspread/service_account.json'
# 3. run the following:
import gspread
_gc = gspread.service_account() # need a key file to access the account (step 2)
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
gmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

In [3]:
gmetadata

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),Pupil tracking pre-processing,Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Notes
1,mouse496220,audio_vis1_2020-06-10_14-54-43,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
2,mouse496220,audio_vis2_2020-06-11_11-42-47,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,29,None,None,None,None,None,None,None,None
3,mouse496220,audio_vis3_2020-06-16_10-35-57,run/resting,sensory,black,whitenoise,5,250,,,20,None,None,None,None,None,None,None,None,None
4,mouse496220,audio_vis4_2020-06-18_13-49-17,run/resting,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
5,mouse521885,audio_vis1_2020-07-08_12-37-58,awake/ISO,sensory,black/white,whitenoise/10000,[3.5 4.5],250,,,50,6,None,None,None,None,None,None,None,None
6,mouse521885,estim1_2020-07-09_14-23-49,awake/ISO,electrical,N/A,N/A,[3.5 4.5],0.2,20/50/100,M2,60,"6,7,8,10,11,12,13,14,15,16,17,18,21",None,None,None,None,None,None,None,None
7,mouse521886,audio_vis1_2020-07-15_13-28-29,awake/ISO,sensory,black/white,whitenoise/10000,[3.5 4.5],250,,,60,None,None,None,None,None,None,None,None,None
8,mouse521886,estim1_2020-07-16_13-37-02,awake/ISO/recovery,electrical,N/A,N/A,[3.5 4.5],0.2,20/50/100,M2,100,"10,11,12,13,14,15,16,17,18,19",None,None,None,None,None,None,None,None
9,mouse521887,audio1_2020-07-29_09-13-05,awake/awake/awake/ISO/ISO/ISO_low/recovery/rec...,sensory,N/A,whitenoise/10000,2.5,250,,,100,"7,18",None,None,None,None,None,None,None,None
10,mouse521887,estim1_2020-07-30_11-25-05,awake/awake,electrical,N/A,N/A,[3.5 4.5],0.2,20,M2,100,"4,7,9,11,12,13,14,15,18",None,None,None,None,None,None,None,None


In [4]:
epoch_cms = {
    'pre' : cm.Reds,
    'iso_high' : cm.PuOr,
    'iso_low' : cm.PuOr_r,
    'early_recovery': cm.Blues,
    'late_recovery' : cm.Greens
}

bois = {
    'low' : (0, 10),
    'delta' : (0, 4),
    'theta' : (5, 8),
    'alpha' : (8, 13),
    'beta' : (15, 30),
    'gamma_low' : (30, 60),
    'gamma_high' : (60, 100),
    'high' : (10, 100)
}

running_threshold = 0.5

In [5]:
# mouse = 'mouse569069'
display(gmetadata[gmetadata.mouse_name.isin(['mouse551397', 'mouse569062', 'mouse569069', 'mouse569072'])])
# expt = 'estim_vis2_2021-03-12_10-52-44'

mes = [
    ('mouse551397', 'estim_vis_2021-02-11_10-45-23'),# ('mouse569062', 'estim_vis_2021-02-18_11-17-51'),
    ('mouse569069', 'estim_vis2_2021-03-12_10-52-44'),# ('mouse569072', 'estim_vis_2021-04-22_10-26-58')
]

op = 'anton_proposal_data/'

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),Pupil tracking pre-processing,Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Notes
27,mouse551397,estim_vis_2021-02-11_10-45-23,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,10/30/50,M2,120,"26,28,29","F,B",X,X,,X,"[ 372,142,395], [ 414,266,432]","MOs1, MOs6a",ISO kept ~1%. White circles for the vis stim. ...
28,mouse569062,estim_vis_2021-02-18_11-17-51,awake/ISO/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,15/35/60,M2,120,"6,13","F,B,D",X,X,,X,"[370,146,387], [ 428,300,434]","MOs1, ccg",ISO kept ~1%. White circles for the vis stim. ...
30,mouse569069,estim_vis1_2021-03-11_11-02-08,awake/awake/awake/awake,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,50/60/80,M2,120,"7,8,9,10,11,12,13,14","F,B,C",X,X,,X,"[293,156,456], [331,266,505]","MOs1,PL5",control exp in awake. Something happen to the ...
31,mouse569069,estim_vis2_2021-03-12_10-52-44,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,20/40/70,M2,120,,"F,B,C",X,X,,X,"[293,156,456], [331,266,505]","MOs1,PL5",ISO kept ~1%.
39,mouse569072,estim_vis_2021-04-22_10-26-58,awake/ISO,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,30/50/70,SS-cortex,120,"3,4,5,7,8,9,10,11,13","F,B,C",X,X,,,,,ISO kept ~1%. EEG filter at 0.1Hz. EEG has hi...


In [6]:
def load_visual_responses(mouse, expt, channel=4, sample_rate=208):
    data_folder = f"../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/"

    # load experiment metadata and eeg data
    exp = EEGexp(data_folder, preprocess=False)
    eegdata = exp.load_eegdata(frequency=sample_rate, return_type='pd')
    # common average rereferencing
    eegdata = (eegdata.T - eegdata.T.mean()).T
    timestamps = eegdata.index

    # load other data (running, iso etc)
    print('Loading other data...')
    running_speed = exp.load_running(return_type='pd')

    stim_log = pd.read_csv(exp.stimulus_log_file)
    stim_log.rename_axis(index='stim_id', inplace=True)

    idx = stim_log.reset_index().set_index('onset')
    idx.index = idx.index - 1
    idx = idx.reindex(timestamps.rename('onset'), method='ffill', limit=sample_rate*6).reset_index()

    def _reset_index_time(df):
        df['time'] = df.onset
        df['onset'] = (df.onset - df.onset.iloc[0] - 1).round(4)
        return df
    idx = idx.groupby('stim_id').apply(_reset_index_time).drop(['offset', 'duration'], axis=1).set_index('time').rename(columns={'onset':'trial_time'})
    
    # add running speed information to idx
    idx = idx.join(running_speed.reindex(idx.index, method='nearest'))
    # compute mean running speed for trial
    idx['running_speed'] = idx.stim_id.map(idx.groupby('stim_id').apply(lambda df: df.running_speed.mean()))
    
    idx['condition'] = idx.running_speed.map(lambda x: 'running' if x>running_threshold else 'resting')
    idx.loc[idx.index[idx.sweep==1], 'condition'] = 'anesthetized'
    idx.loc[idx.index[(idx.sweep>1)&(idx.condition=='resting')], 'condition'] = 'recovery'
#     idx.loc[idx.index[idx.sweep>1], 'condition'] = 'recovery'
    
    eegdata = eegdata.loc[idx.index]
    eegdata.index = pd.MultiIndex.from_frame(idx[['stim_type', 'parameter', 'sweep', 'stim_id', 'condition', 'running_speed', 'trial_time']])
    eegdata = eegdata.sort_index()
    
    vis_resp = eegdata.xs('circle', level='stim_type')[channel].unstack().T.dropna()
    vis_resp = vis_resp.rename(
        columns={i:x for i, x in enumerate(gmetadata[
            (gmetadata.mouse_name==mouse)&(gmetadata.exp_name==expt)
        ].iloc[0]['brain states'].split('/'))},
        level=1
    ).droplevel([0, -1], axis=1)
    
    return vis_resp

In [7]:
if not path.exists(f'{op}responses.pkl'):
    responses = {}
    for mouse, expt in tqdm(mes):
        responses[mouse] = load_visual_responses(mouse, expt)
    responses = pd.concat(responses, names=['mouse', 'time']).unstack(0).swaplevel(0, -1, axis=1).sort_index(axis=1)
    responses.to_pickle(f'{op}responses.pkl')
else:
    responses = pd.read_pickle(f'{op}responses.pkl')

responses = responses.dropna(axis=1, how='all').swaplevel(1, 2, axis=1).sort_index(axis=1).drop('recovery', level='condition', axis=1)
responses = responses[[m[0] for m in mes]]
responses.columns = responses.columns.remove_unused_levels()

# Quantify some metrics on the responses

## 1. Delayed inhibition during waking

In [8]:
for mouse in responses.columns.levels[0]:
    f, ax = plt.subplots(figsize=(8, 3), tight_layout=True)

    responses[mouse].dropna().groupby('condition', axis=1).mean().plot(
        ax=ax, yerr=responses[mouse].dropna().groupby('condition', axis=1).sem(),
        alpha=0.15
    )
    # ax.set_xlim(-1, 0.5)
    ax.axvspan(0, 0.25, color=cm.Greys(0.4, 0.3))
    ax.legend(fontsize=8)
    ax.set_ylabel('EEG response ($\mu V$)')
    ax.set_title(f'Delayed inhibition during waking ({mouse})');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Differences at short times post stimulation

In [9]:
for mouse in responses.columns.levels[0]:
    f, ax = plt.subplots(figsize=(8, 3), tight_layout=True)

    responses[mouse].dropna().groupby('condition', axis=1).mean().plot(
        ax=ax, yerr=responses[mouse].dropna().groupby('condition', axis=1).sem(),
        alpha=0.5
    )
    ax.set_xlim(-0.25, 0.55)
    ax.axvspan(0, 0.25, color=cm.Greys(0.4, 0.3))
    ax.legend(fontsize=8)
    ax.set_ylabel('EEG response ($\mu V$)')
    ax.set_title(f'Differences between 50 - 150 ms ({mouse})');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3. Spectral differentiation TODO compute for mean responses in epochs

In [10]:
# # spectran differentiation on a single electrode channel (channel 4, visual area)

# def compute_differentiation(state_length, window_length, t_start, t_end):
#     data = vis_resp.loc[t_start:t_end].values.T[:, np.newaxis, :int(sample_rate*window_length)]
#     dfn = differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=200)
#     dfn = pd.Series(np.median(dfn, axis=1), index=vis_resp.columns, name='spectral differentiation')
#     return dfn

# def plot_differentiation(state_length, window_length, t_start, t_end, ax):
#     dfn = compute_differentiation(state_length, window_length, t_start, t_end)
#     sns.boxplot(x='sweep', y='spectral differentiation', data=dfn.reset_index(), order=['ISO', 'awake', 'recovery'], ax=ax)
#     ax.set_title(f'differentiation ({mouse}) {t_start, window_length}')
#     return


# f, axes = plt.subplots(1, 4, figsize=(14, 3), tight_layout=True, sharex=True, sharey=True)

# plot_differentiation(state_length=0.02, window_length=0.1, t_start=0, t_end=2, ax=axes[0])
# plot_differentiation(state_length=0.02, window_length=0.6, t_start=0, t_end=2, ax=axes[1])
# plot_differentiation(state_length=0.02, window_length=1.2, t_start=0, t_end=2, ax=axes[2])
# plot_differentiation(state_length=0.02, window_length=0.6, t_start=-1, t_end=0, ax=axes[3])


# f, axes = plt.subplots(1, 4, figsize=(14, 3), tight_layout=True, sharex=True, sharey=True)

# plot_differentiation(state_length=0.03, window_length=0.24, t_start=0, t_end=2, ax=axes[0])
# plot_differentiation(state_length=0.03, window_length=0.24, t_start=0.2, t_end=2, ax=axes[1])
# plot_differentiation(state_length=0.03, window_length=0.24, t_start=0.4, t_end=2, ax=axes[2])
# plot_differentiation(state_length=0.03, window_length=0.24, t_start=-1, t_end=0, ax=axes[3])

## 4. Spectral analysis

In [11]:
# # spectral analysis
# # get the rolling short time fourier transform for mua
# def _get_stft_as_df(freqs, times, data):
#     return pd.DataFrame(data, freqs, times).abs()**2

# stft = pd.concat({
#     mouse:pd.concat({
#         c : _get_stft_as_df(
#             *sp.signal.stft(responses[mouse][c], nperseg=50, fs=208)
#         ) for c in responses[mouse].columns
#     }, axis=1) for mouse in responses.columns.levels[0]
# }, axis=1, names=['mouse', 'stim_id', 'condition', 'sweep', 'time'])

# band_powers = {}
# for band in bois.keys():
#     if band in ['high', 'low']:
#         continue
#     p = stft.loc[slice(*bois[band])].mean().unstack(list(np.arange(stft.columns.nlevels-1, dtype=int)))
#     p = p / p.loc[:1].mean()
#     band_powers[band] = p.rolling(10, center=True).mean()
# band_powers = pd.concat(band_powers, axis=1).dropna(axis=1, how='all')

# band_powers_mean = band_powers.groupby(level=['mouse', 0], axis=1).apply(lambda df: df.groupby('condition', axis=1).median()).dropna()
# band_powers_sd = band_powers.groupby(level=['mouse', 0], axis=1).apply(lambda df: df.groupby('condition', axis=1).sem()).dropna()
# band_powers_mean.index = band_powers_mean.index - 1
# band_powers_sd.index = band_powers_mean.index

# f, axes = plt.subplots(
#     len(mes), 5, figsize=(15, 2*len(mes)),
#     tight_layout=True, sharex=True
# )
# axes = {
#     mouse[0] : {
#         b:axes[j, i] for i, b in enumerate(band_powers_mean.columns.levels[1])
#     } for j, mouse in enumerate(mes)
# }
# band_powers_mean.groupby(level=['mouse', 1], axis=1).apply(
#     lambda df: df[df.name].plot(
#         ax=axes[df.name[0]][df.name[1]], legend=False,
#         yerr=band_powers_sd[df.name], alpha=0.5
#     )
# )
# for mouse, axe in axes.items():
#     for b, ax in axe.items():
#         ax.legend(fontsize=6)
#         ax.set_title(b)
#         ax.set_xlabel('time (s)')
#         ax.set_ylabel(mouse)
#         ax.axvline(0, c='k', lw=0.3)
# f.suptitle('spectral analysis');

## 4. Number of PCs

In [12]:
responses.groupby('mouse', axis=1).apply(lambda df: display(df.groupby('condition', axis=1).size()))

condition
anesthetized    120
resting         109
running          45
dtype: int64

condition
anesthetized    120
resting          80
running          72
dtype: int64

""


In [60]:
def plot_df(df, x, y, style='box', palette=None):
    if style=='box':
        sns.boxplot(x=x, y=y, data=df.reset_index(), palette=palette, medianprops=dict(color='k'))
    else:
        df.unstack().plot(ax=ax, marker='o', markerfacecolor='none', color='grey', legend=False, ms=4)
        ax.errorbar(df.unstack(0).mean().index, df.unstack(0).mean(), df.unstack(0).sem())

In [46]:
def get_npcs(df, pev=0.99, t=(0, 0.1)):
    pca = PCA(n_components=pev)
    df = ((df.loc[t[0]:t[1]].T - df.loc[t[0]:t[1]].mean(1))/df.loc[t[0]:t[1]].std(1))
    pca.fit(df, )
    return pca.n_components_

In [63]:
method = 'bootstrap'
nreps = 100
pev = 0.99
plotstyle = 'box'

pal = {'anesthetized':cm.Blues(0.7, 0.9), 'resting':cm.Reds(0.7, 0.9), 'running':cm.Greens(0.7, 0.9)}
pal = {'anesthetized':'b', 'resting':'r', 'running':'g'}
for t in [(0, 0.1)]:
    if method=='bootstrap':
        npcs = pd.concat([
            responses.groupby('mouse', axis=1).apply(lambda df: df.groupby('condition', axis=1).apply(
                lambda d: get_npcs(d.iloc[:, np.random.choice(len(d.columns), 45, replace=True)], pev=pev, t=t)
            )) for i in range(nreps)
        ], axis=1, ignore_index=True).unstack().rename('npcs')
    elif method=='all':
        npcs = responses.groupby(['mouse', 'condition'], axis=1).apply(lambda d: get_npcs(d, pev=pev, t=t)).rename('npcs')

    f, ax = plt.subplots(figsize=(4.8, 3.), tight_layout=True)
    plot_df(npcs.swaplevel().loc[['resting', 'running', 'anesthetized']], 'condition', 'npcs', style=plotstyle, palette=pal)
    ax.set_xlabel('')
    ax.set_ylabel(f'number of PCs\n({pev*100:.0f}% variance)')
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['Resting', 'Running', 'Anesthetized'])
    print(f'{t[0]} to {t[1]} s', method)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 to 0.1 s bootstrap


In [29]:
npcs.unstack().median()

condition
anesthetized    11.0
resting         13.0
running         13.5
dtype: float64

## 5. AUC

In [53]:
method = 'bootstrap' # or 'bootstrap' or 'all'
nreps = 100
plotstyle = 'box' # 'box' or ''
dt = np.diff(responses.index).mean()

if method=='all':
    # one shot auc for whatever number of trials
    auc = responses.groupby(['mouse', 'condition'], axis=1).apply(lambda df: df.mean(1).abs().loc[0:0.1].sum()).rename('auc').swaplevel()*dt
else:
    # bootstrapped auc
    auc = pd.concat([
        responses.groupby('mouse', axis=1).apply(lambda df: df.groupby('condition', axis=1).apply(
            lambda d: d.iloc[:, np.random.choice(len(d.columns), 45, replace=True)].mean(1).abs().loc[0:0.1].sum()
        )) for i in range(nreps)
    ], axis=1, ignore_index=True).unstack().rename('auc').swaplevel()*dt
display(auc)

f, ax = plt.subplots(figsize=(4.8, 3.), tight_layout=True)
plot_df(auc.loc[['resting', 'running', 'anesthetized']], 'condition', 'auc', style=plotstyle, palette=pal)
ax.set_ylabel('AUC Abs. Ch4 Voltage ($\mu Vs$)')
ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['Resting', 'Running', 'Anesthetized'])
print(method, t)

condition        
anesthetized  0      0.470130
resting       0      0.767321
running       0      1.019360
anesthetized  1      0.643218
resting       1      0.643719
                       ...   
              198    0.419580
running       198    0.808974
anesthetized  199    0.880793
resting       199    0.458290
running       199    0.871698
Name: auc, Length: 600, dtype: float64

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

bootstrap (0, 0.1)
